<a href="https://colab.research.google.com/github/vtecftwy/utseus-dives/blob/main/nbs/dive_2.2_rag_w_langchain.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Build a multi task agent with Langchain

Inspired by LangChain supervisor agent tutorial [here](https://docs.langchain.com/oss/python/langchain/supervisor)

## 0. Installs, Imports, Utility Fctn and Setup

In [1]:
import faiss
import os
import textwrap
from dotenv import load_dotenv
from langchain_community.docstore.in_memory import InMemoryDocstore
from langchain_community.vectorstores import FAISS
from langchain_core.vectorstores import InMemoryVectorStore
from langchain_openai import OpenAIEmbeddings
from pathlib import Path
from pprint import pprint

In [2]:
def show_attr(o):
    for a in o.__dict__:
        if not '_' in a:
            print(f"{a:10s}: \t{getattr(o, a)}")

def printmd(*txt):
    for t in txt:
        display(Markdown(t))

def setup_clean_proxy():
    """Clear problematic proxy vars and set clean ones"""
    env_backup = {}
    proxy_vars = ['http_proxy', 'https_proxy', 'HTTP_PROXY', 'HTTPS_PROXY', 'no_proxy', 'NO_PROXY']
    for var in proxy_vars:
        if var in os.environ:
            env_backup[var] = os.environ[var]
            del os.environ[var]
    
    os.environ['HTTPS_PROXY'] = 'http://127.0.0.1:16005'
    os.environ['HTTP_PROXY'] = 'http://127.0.0.1:16005'
    return env_backup

# Call at start
env_backup = setup_clean_proxy()

In [3]:
load_dotenv()
api_key = os.getenv("OPENAI_API_KEY")

## Define model

In [4]:
from langchain.chat_models import init_chat_model

model = init_chat_model("gpt-4.1-mini")

## Define tools

In [5]:
from langchain.tools import tool

@tool
def create_calendar_event(
    title: str,
    start_time: str,       # ISO format: "2024-01-15T14:00:00"
    end_time: str,         # ISO format: "2024-01-15T15:00:00"
    attendees: list[str],  # email addresses
    location: str = ""
) -> str:
    """Create a calendar event. Requires exact ISO datetime format."""
    # Stub: In practice, this would call Google Calendar API, Outlook API, etc.
    return f"Event created: {title} from {start_time} to {end_time} with {len(attendees)} attendees"


@tool
def send_email(
    to: list[str],  # email addresses
    subject: str,
    body: str,
    cc: list[str] = []
) -> str:
    """Send an email via email API. Requires properly formatted addresses."""
    # Stub: In practice, this would call SendGrid, Gmail API, etc.
    return f"Email sent to {', '.join(to)} - Subject: {subject}"


@tool
def get_available_time_slots(
    attendees: list[str],
    date: str,  # ISO format: "2024-01-15"
    duration_minutes: int
) -> list[str]:
    """Check calendar availability for given attendees on a specific date."""
    # Stub: In practice, this would query calendar APIs
    return ["09:00", "14:00", "16:00"]

## Create specialized sub-agents

### Create a calendar agent

The calendar agent understands natural language scheduling requests and translates them into precise API calls. It handles date parsing, availability checking, and event creation.

In [6]:
from langchain.agents import create_agent

CALENDAR_AGENT_PROMPT = (
    "You are a calendar scheduling assistant. "
    "Parse natural language scheduling requests (e.g., 'next Tuesday at 2pm') "
    "into proper ISO datetime formats. "
    "Use get_available_time_slots to check availability when needed. "
    "Use create_calendar_event to schedule events. "
    "Always confirm what was scheduled in your final response."
)

calendar_agent = create_agent(
    model,
    tools=[create_calendar_event, get_available_time_slots],
    system_prompt=CALENDAR_AGENT_PROMPT,
)

Test the calendar agent

In [7]:
query = "Schedule a team meeting next Tuesday at 2pm for 1 hour"

for step in calendar_agent.stream(
    {"messages": [{"role": "user", "content": query}]}
):
    for update in step.values():
        for message in update.get("messages", []):
            message.pretty_print()

================================== Ai Message ==================================
Tool Calls:
  get_available_time_slots (call_rLaOisar24xZWEAj0mU9Tolu)
 Call ID: call_rLaOisar24xZWEAj0mU9Tolu
  Args:
    attendees: ['team']
    date: 2024-04-02
    duration_minutes: 60
================================= Tool Message =================================
Name: get_available_time_slots

["09:00", "14:00", "16:00"]
================================== Ai Message ==================================
Tool Calls:
  create_calendar_event (call_ktM6Jb0nUcdQeHxmtIKwVKox)
 Call ID: call_ktM6Jb0nUcdQeHxmtIKwVKox
  Args:
    title: Team Meeting
    start_time: 2024-04-02T14:00:00
    end_time: 2024-04-02T15:00:00
    attendees: ['team']
================================= Tool Message =================================
Name: create_calendar_event

Event created: Team Meeting from 2024-04-02T14:00:00 to 2024-04-02T15:00:00 with 1 attendees
================================== Ai Message =========================

### Create an email agent

In [8]:
EMAIL_AGENT_PROMPT = (
    "You are an email assistant. "
    "Compose professional emails based on natural language requests. "
    "Extract recipient information and craft appropriate subject lines and body text. "
    "Use send_email to send the message. "
    "Always confirm what was sent in your final response."
)

email_agent = create_agent(
    model,
    tools=[send_email],
    system_prompt=EMAIL_AGENT_PROMPT,
)

Test the email agent

In [9]:
query = "Send the design team a reminder about reviewing the new mockups"

for step in email_agent.stream(
    {"messages": [{"role": "user", "content": query}]}
):
    for update in step.values():
        for message in update.get("messages", []):
            message.pretty_print()

================================== Ai Message ==================================
Tool Calls:
  send_email (call_mkqWucrZ2S1yxXJfgUdZz1PI)
 Call ID: call_mkqWucrZ2S1yxXJfgUdZz1PI
  Args:
    to: ['design-team@example.com']
    subject: Reminder: Review of New Mockups
    body: Hello Design Team,

This is a friendly reminder to review the new mockups at your earliest convenience. Please share your feedback or any suggestions you might have.

Thank you for your attention to this.

Best regards,
[Your Name]
================================= Tool Message =================================
Name: send_email

Email sent to design-team@example.com - Subject: Reminder: Review of New Mockups
================================== Ai Message ==================================

I have sent a reminder email to the design team about reviewing the new mockups. Is there anything else you would like to do?


## Wrap sub-agents as tools

Now wrap each sub-agent as a tool that the supervisor can invoke. This is the key architectural step that creates the layered system. The supervisor will see high-level tools like “schedule_event”, not low-level tools like “create_calendar_event”.

In [10]:
@tool
def schedule_event(request: str) -> str:
    """Schedule calendar events using natural language.

    Use this when the user wants to create, modify, or check calendar appointments.
    Handles date/time parsing, availability checking, and event creation.

    Input: Natural language scheduling request (e.g., 'meeting with design team
    next Tuesday at 2pm')
    """
    result = calendar_agent.invoke({
        "messages": [{"role": "user", "content": request}]
    })
    return result["messages"][-1].text


@tool
def manage_email(request: str) -> str:
    """Send emails using natural language.

    Use this when the user wants to send notifications, reminders, or any email
    communication. Handles recipient extraction, subject generation, and email
    composition.

    Input: Natural language email request (e.g., 'send them a reminder about
    the meeting')
    """
    result = email_agent.invoke({
        "messages": [{"role": "user", "content": request}]
    })
    return result["messages"][-1].text

##  Create the supervisor agent

In [11]:
SUPERVISOR_PROMPT = (
    "You are a helpful personal assistant. "
    "You can schedule calendar events and send emails. "
    "Break down user requests into appropriate tool calls and coordinate the results. "
    "When a request involves multiple actions, use multiple tools in sequence."
)

supervisor_agent = create_agent(
    model,
    tools=[schedule_event, manage_email],
    system_prompt=SUPERVISOR_PROMPT,
)

## Use the supervisor

### Simple single-domain request

In [12]:
query = "Schedule a team standup for tomorrow at 9am"

for step in supervisor_agent.stream(
    {"messages": [{"role": "user", "content": query}]}
):
    for update in step.values():
        for message in update.get("messages", []):
            message.pretty_print()

================================== Ai Message ==================================
Tool Calls:
  schedule_event (call_OcAGoM5Q28k8HxiHx0RwAcMT)
 Call ID: call_OcAGoM5Q28k8HxiHx0RwAcMT
  Args:
    request: team standup tomorrow at 9am
================================= Tool Message =================================
Name: schedule_event

Your team standup is scheduled for tomorrow at 9:00 AM and will last for 30 minutes.
================================== Ai Message ==================================

The team standup is scheduled for tomorrow at 9:00 AM. Is there anything else you would like to do?


### Complex multi-domain request

In [16]:
query = (
    "Schedule a meeting with the design team next Tuesday at 2pm for 1 hour, "
    "and send them an email reminder about reviewing the new mockups."
)

for step in supervisor_agent.stream(
    {"messages": [{"role": "user", "content": query}]}
):
    for update in step.values():
        for message in update.get("messages", []):
            message.pretty_print()

================================== Ai Message ==================================
Tool Calls:
  schedule_event (call_f6RhOg9PkjtOu0QlJVUMxaKa)
 Call ID: call_f6RhOg9PkjtOu0QlJVUMxaKa
  Args:
    request: meeting with the design team next Tuesday at 2pm for 1 hour
  manage_email (call_GJQSDuGiVT9lRJFXE2KpBJWh)
 Call ID: call_GJQSDuGiVT9lRJFXE2KpBJWh
  Args:
    request: send an email reminder to the design team about reviewing the new mockups
================================= Tool Message =================================
Name: manage_email

I have sent a reminder email to the design team about reviewing the new mockups. If you need any further assistance, feel free to ask!
================================= Tool Message =================================
Name: schedule_event

I have scheduled the meeting with the design team next Tuesday, April 23rd, from 2pm to 3pm.
================================== Ai Message ==================================

The meeting with the design team is sched

### Complex multi-domain request 2

In [14]:
query = (
    "Schedule a meeting with the design team next Tuesday at 2pm for 1 hour, "
    "and send them an email reminder about reviewing the new mockups."
    "Then schedule a meeting with John next Friday at 3 pm for 30 minutes. If not available, pick the next available slot. "
    "And send a email to John about the meeting, telling him that he has to prepare a presentation for Project x."
)

for step in supervisor_agent.stream(
    {"messages": [{"role": "user", "content": query}]}
):
    for update in step.values():
        for message in update.get("messages", []):
            message.pretty_print()

================================== Ai Message ==================================
Tool Calls:
  schedule_event (call_VNbuw5aWCc8QiYiNgFZltzed)
 Call ID: call_VNbuw5aWCc8QiYiNgFZltzed
  Args:
    request: meeting with the design team next Tuesday at 2pm for 1 hour
  schedule_event (call_Xfc5fCmLrEKnJpaT6RgP5G10)
 Call ID: call_Xfc5fCmLrEKnJpaT6RgP5G10
  Args:
    request: meeting with John next Friday at 3pm for 30 minutes, if not available pick the next available slot
================================= Tool Message =================================
Name: schedule_event

I have scheduled your meeting with the design team next Tuesday, June 11th, from 2pm to 3pm. If you need any changes or additional details, please let me know!
================================= Tool Message =================================
Name: schedule_event

John is not available next Friday at 3pm. I have scheduled the meeting with John on next Friday at 2pm for 30 minutes instead.
================================== 